In [2]:
import pandas as pd
import matplotlib as mp
import numpy as np
import math as m

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb

%matplotlib inline

#Root de archivos de datos
root_dir = '../data/'
group=['ref_hash','window_nr']

---
## Read de Features

In [3]:
features = pd.read_csv('training_set.csv').set_index(group)

In [4]:
features.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event,wifi,timeToClick_mean,amount_dif_advertisers,amount_installs,secs_to_next_install_mean,last_event,before_last_event,before_before_last_event
ref_hash,window_nr,,,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,88625.0,216.0,0.0,0.0,False,3.0,0.0,0.0,False,198.0,0.0,0.0,7632.0,0.0,0.0,0.0
4441121667607578179,1,111.0,29052.0,1188.0,0.0,640.0,True,5.0,0.0,0.0,False,198.0,0.0,0.0,7632.0,0.0,0.0,0.0
7721769811471055264,1,281.0,333.0,908.0,23.0,218.0,True,3.0,99.0,196475.0,False,198.0,0.0,0.0,7632.0,2.0,2.0,2.0
1258642015983312729,1,8365.0,121224.0,12.0,0.0,0.0,False,1.0,0.0,0.0,False,198.0,0.0,0.0,7632.0,0.0,0.0,0.0
6707090658317158573,1,453.0,4.0,516.0,35.0,171.0,True,4.0,0.0,0.0,False,198.0,0.0,0.0,7632.0,0.0,0.0,0.0


In [5]:
targets = pd.read_csv('targets.csv').set_index(group)

In [6]:
targets.isnull().sum()

target_st    0
target_sc    0
dtype: int64

In [7]:
training_set = features.merge(targets, how='inner', left_index=True,right_index=True)

In [8]:
training_set.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event,wifi,timeToClick_mean,amount_dif_advertisers,amount_installs,secs_to_next_install_mean,last_event,before_last_event,before_before_last_event,target_st,target_sc
ref_hash,window_nr,,,,,,,,,,,,,,,,,,,
4881735474151208235,1,8033.0,0.0,0.0,0.0,0.0,False,0.0,2.0,23418.0,False,198.000,0.0,0.0,7632.0,211.0,7.0,0.0,259200.0,259200.0
4679881814211948892,1,45.0,156639.0,5.0,0.0,0.0,False,2.0,0.0,0.0,False,25.335,1.0,0.0,7632.0,0.0,0.0,0.0,63602.0,259200.0
4177895333092974918,1,3990.0,119428.0,15.0,0.0,0.0,False,2.0,0.0,0.0,False,198.000,0.0,0.0,7632.0,0.0,0.0,0.0,259200.0,259200.0
4312326900897087220,1,4703.0,10305.0,53.0,0.0,22.0,True,4.0,0.0,0.0,False,198.000,0.0,0.0,7632.0,0.0,0.0,0.0,8056.0,259200.0
4911541169216784265,2,7894.0,0.0,0.0,0.0,0.0,False,0.0,14.0,69048.0,False,186.000,0.0,1.0,40.0,14.0,13.0,19.0,259200.0,259200.0


In [9]:
x_st = training_set[[x for x in training_set.columns if x not in targets.columns]]
y_st = training_set['target_st']

x_sc = training_set[[x for x in training_set.columns if x not in targets.columns]]
y_sc = training_set['target_sc']

---
## Training 

In [10]:
xtrain_st, xtest_st, ytrain_st, ytest_st = train_test_split(x_st, y_sc, train_size=0.7)
xtrain_sc, xtest_sc, ytrain_sc, ytest_sc = train_test_split(x_sc, y_sc, train_size=0.7)

In [34]:
model_st = KNeighborsRegressor(n_neighbors=10)
model_sc = KNeighborsRegressor(n_neighbors=10)

model_st.fit(xtrain_st, ytrain_st)
model_sc.fit(xtrain_sc, ytrain_sc)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                    weights='uniform')

In [35]:
# 1. Solo Features Nico: 
#    Error St.: 
#    Error Sc.: 
# 2. FEatures Nico y Manu: 
#    Error St.: 
#    Error Sc.: 

ypred_st = model_st.predict(xtest_st)
print("Error St.:", m.sqrt(mean_squared_error(ypred_st,ytest_st)))

ypred_sc = model_sc.predict(xtest_sc)
print("Error Sc.:", m.sqrt(mean_squared_error(ypred_sc,ytest_sc)))

Error St.: 52729.52134344401
Error Sc.: 52324.56730562874


---
## Prediccion

In [14]:
# Labels a submitir con las predicciones

to_predict = pd.read_csv(root_dir + 'target_competencia_ids.csv',usecols=['ref_hash'])
to_predict.head()

,ref_hash
0,1000169251625791246_sc
1,1000169251625791246_st
2,1000395625957344683_sc
3,1000395625957344683_st
4,1003027494996471685_sc


In [25]:
new_features = features.groupby('ref_hash').mean()

In [26]:
X = new_features

In [27]:
y_st = model_st.predict(X)
y_sc = model_sc.predict(X)

In [28]:
results_st = X.reset_index()['ref_hash'].transform(lambda x: str(x) + '_st').rename('ref_hash').to_frame()
results_st['obj'] = y_st

results_sc = X.reset_index()['ref_hash'].transform(lambda x: str(x) + '_sc').rename('ref_hash').to_frame()
results_sc['obj'] = y_sc

results = results_st.append(results_sc)

In [29]:
targets_mean = targets.reset_index()
targets_mean = targets_mean.loc[~targets_mean['ref_hash'].isin(X.reset_index()['ref_hash'])].groupby('ref_hash')[['target_st','target_sc']].mean().reset_index()

t_st = targets_mean['ref_hash'].transform(lambda x: str(x) + '_st').rename('ref_hash').to_frame()
t_st['obj'] = targets_mean['target_st']

t_sc = targets_mean['ref_hash'].transform(lambda x: str(x) + '_sc').rename('ref_hash').to_frame()
t_sc['obj'] = targets_mean['target_sc']

r = t_st.append(t_sc)

In [30]:
results = results.append(r)

In [31]:
to_predict.merge(results, on='ref_hash', how='left')['obj'].count()

8056

In [32]:
#Armado de File Final
max_secs = 3*24*60*60
to_predict.merge(results, on='ref_hash', how='left').fillna(max_secs).to_csv('predictions_knn.csv', header=True, index=False)

In [33]:
to_predict.count()

ref_hash    8074
dtype: int64